## **Feature Preprocessing and Generation with Respect to Models**

## イントロダクション

### 前処理と特徴量生成について

前処理も特徴量生成もどのように処理するかは用いるモデルタイプに依存する。
以下、具体例を示す。

- カテゴリカル変数の前処理
    - 線形モデルは One-hot Encoding などが有効
    - ツリーベースモデルでは不要
    
- トレンドを反映させるための特徴量生成
    - 線形モデルでは前週の売上を特徴量として足す
    - ツリーベースでは Window で Target Encoding したものを特徴量として足す

## 数値変数（Numerical Features）

### 前処理

#### スケーリング（Scaling）

- ツリーベースモデル
    - 値のスケールに影響されない
- 非ツリーベースモデル
    - 値のスケールに影響を受ける
    - kNN の例にして考えるとスケールが予測精度に与える影響を理解しやすい
    
- 具体的な手法
    - sklearn.preprocessing.MinMaxScaler()
        - 指定した最小値と最大値の範囲に値をスケーリングする（デフォルトでは0~1の範囲）
    - sklearn.preprocessing.StandardScaler()
        - mean を引いて std で割る（平均0・標準偏差を1にする）
        
- kNN においてスケーリングの考え方を応用して、より重要だと思われる特徴量のスケールを大きくすることで、モデルへのその特徴量の影響度合いを（意図的に）大きくすることができる。

#### 外れ値（Outliers）

- 線形モデルは外れ値に影響を受けやすい
- これを避けるためのアプローチがいくつかある。
    - 1. パーセンタイルをもとに値をクリッピングする
    - 2. ランク変換（Rank Transformation）を行う。値のマッピングを行うイメージか。
        - ```rank([-100, 0, 1e5])==[0, 1, 2]```  
          ```rank([1000, 1, 10]) = [2, 0, 1]```  
        
#### その他

- 非ツリーベースモデル（特にNN）でしばしば有効な前処理
    - ログ変換（Log Transform): np.log(1+x)
    - 指数変換(Rasing to the power<1): np.sqrt(x + 2/3)
- これらは過分散を抑制する一方で、0に近い値の差を大きくする
- 様々な前処理を行って学習を行ったモデルを組み合わせることはとても有効になりうる。

### 特徴量生成

- 特徴量生成とは
    - 特徴量やタスクに関する知識を利用して新しい特徴量を作り出すこと。
    - モデルの学習をよりシンプルかつ効果的にするのに役立つ。
    - 事前知識や EDA を用いたり、仮説をもとに検証したり、あるいは直感によって新しい特徴量を引き出す。
    
- 数値変数に関する特徴量生成について
    - 加減乗除・交互作用による特徴量生成は線形モデルのみならずツリーベースモデルに対しても有効である。
    - ツリーベースモデルに掛け算や割り算による良い特徴量を追加することで、よりロバストで木の数を抑えたモデルを作ることができる。
    - 直感に基づくザックリした特徴量も有効になりうる。

## カテゴリカル・順序変数（Categorical and Oridinal Features）

### 前処理

- カテゴリカル変数の中でも、数値の順序に意味があるものを順序変数（あるいは Order Categorical Features）と呼ぶ。

#### Label Encoding
- 順序変数を数値に変換する手法
    - ツリーベースモデルでは特徴量を分割できるため有用
    - 一方で、非ツリーベースモデルでは有効には働かない。
- 実装方法
    - sklearn.preprocessing.LabelEncoder: アルファベット順にソートされ変換される
    - pandas.factorize: 出現順に順次変換される
    
#### Frequency Encoding
- カテゴリの出現頻度に応じて値をマッピングする
- ```encoding = titanic.groupby('Embarked').size()```  
  ```encoding = encoding/len(titanic)```  
  ```titanic['enc'] = titanic.Emberked.map(encoding)```  
- これは値の分布に関する情報を持つので、線形モデルやツリーモデルで有効に働きうる。
    - 線形モデルにとっては、もしカテゴリの頻度が目的変数と相関があれば有効になる
    - ツリーベースモデルにとっても、同様の理由でより少ない分割で済むようになりうる。
- 複数のカテゴリが同じ頻度で出現する場合、この方法ではそれらを区別することができない。
    - scipy.stats.rankdata で対処できうる
        
#### One-Hot Encoding
- カテゴリごとに列を作成し、それぞれのカテゴリについて1/0の値で埋める。
- 長所
    - 非ツリーベースモデルで有効に働く
    - 変換後すでにスケーリングされている。
- 短所
    - ツリーベースモデルでは学習の時間を冗長にし、効果が出にくい
- ちなみに、ツリーベースモデルについては、メモリを食うのでスパース行列にして持ったほうが良い。
    - XGBoost や LightGBM ではスパース行列でカテゴリカル変数を保持するように指定できる。


### 特徴量生成

#### 特徴量の交互作用

- 非ツリーベースモデルに対して有効。
- 複数のカテゴリ特徴量の組み合わせを One-Hot の形で持つ

## 日付変数と座標変数（Datetime and Coordinates）

### Date and Time

1. 周期性
    - 曜日・週・月・年・時間・分・秒 などとして利用できる
2. Time since
    - ある日からの経過時間 (Independent)
    - 決まったある日時から/までの時間 (dependent)
3. Difference between dates
    - ある日時から別のある日時までの日時の差
         - churn 予測における、date_diff（購買と電話の日時の差）
         
### Coordinate

- ある地点からの距離
- 集計された統計値を計算する
- ツリーベースモデルを使用するとき、45度（あるいは22.5度）回転させると、決定境界を作らせやすくなる。

## 欠損値処理（Handling Missing Values）

- 数値変数の欠損値には、NaN, Null, -1, 99 などがある。
- ヒストグラムを見ることで、値が欠損値が置き換えられたものだと想定することができる。

### 欠損値へのアプローチ

#### 1. 通常の値の分布外にある値で置き換える（-999, -1, etc.）
- 線形ネットワークモデルには適さない。

#### 2. 平均・中央値で置き換える
- 線形モデルやNNにとっては有効である。
- ツリーベースモデルには適さない。

#### 3. 値を「再構築」する
- 例えば時系列データの場合、欠損値を予測して置き換えることができうる。
- ただし、このようなことができる場合は多くない。


### 欠損値に関する特徴量生成

- 欠損値であるかどうかを示すバイナリの特徴量が作れる
    - ツリーベースモデルやNNで有効（平均や中央値で置き換えるより）
    - ただし、二重に列をつくることにもなる
    
- 欠損値を含む特徴量から別の特徴量を生成する際には、欠損値の置換に特に注意を払う必要がある。
- また 数値変数の欠損を -999 などに補完してからTarget-Encoding などを行ってしまうと、補完した値に平均が引っ張られてしまう。
    - このようなときは欠損値を無視して平均を取ったほうが良い。

- XGBoost は欠損値をそのまま扱え、スコアを大幅に改善することがある。

- 欠損値を異常値として扱うことができる。

- 学習データに現れないカテゴリカル変数のカテゴリはモデルに欠損値として扱われるので、それは Frequency Encoding で置換して対処する方法が考えられる。
    - 学習データとテストデータの両方を見て、その頻度を数え上げる。カテゴリの頻度と目的変数に依存関係があれば良い感じになる。
        - データリークを引き起こすのでは？

## リンク

### Feature preprocessing

- [Preprocessing in Sklearn](https://scikit-learn.org/stable/modules/preprocessing.html)
- [Andrew NG about gradient descent and feature scaling
](https://www.coursera.org/learn/machine-learning/lecture/xx3Da/gradient-descent-in-practice-i-feature-scaling)
- [Feature Scaling and the effect of standardization for machine learning algorithms](http://sebastianraschka.com/Articles/2014_about_feature_scaling.html)

### Feature generation

- [Discover Feature Engineering, How to Engineer Features and How to Get Good at It
](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/)
- [Discussion of feature engineering on Quora](https://www.quora.com/What-are-some-best-practices-in-Feature-Engineering)

---

## **Feature Extraction from Text and Images**

### Text -> Vector

- 1. Bag of words
- 2. Enbeddings (~word2vec)

## Bag of words

### Bag of words
- データから単語ごとに出現回数を数え上げて、文章ごとにそれらを列に配置する。
- sklearn.feature_extranction.text.CountVectorizer で実装可能

### TFiDF

- スケーリングする後処理が必要である。
- Term Frequency: 出現回数ではなく頻度（割合）を計算することで比較可能になる。
    - ```tf = 1/x.sum(axis=1)[:,None]```
    - ```x = x*tf```
- Inverse Document Frequency: ドキュメントの逆数を掛けることで、あるドキュメントにのみ出現する珍しい単語を重視できる。
    - ```idf = np.log(x.shape[0] / (x>0).sum(0)))```
    - ```x = x*idf```
- sklearn.feature_extraction.text.TfidfVectorizer で実装可能

### N-grams

- 単語ごとに特徴量として見なすのではなく、連続する単語ごとに特徴量として見なす。
- unigrams, bigrams, trigrams
- N-grams は単語数がWの場合、W**N この特徴列が生成されることになる。
- sklearn.feature_extraction.text.CountVectorizer: *Ngram_range*, *analyzer*

### Texts preprocessing

- 1. Lowercase
- 2. Lemmatization
- 3. Stemming
- 4. Stopwords

#### 1. Lowercase

- 小文字化: Sunny と sunny を同一のものとして扱いたいため
- sklearn の confing ではデフォルトでこの処理を行ってくれる

#### 2. Lemmatization

- 例）: democracy, democratic, and democratization -> democracy
- 事前知識（辞書）を用いて中心語を類推する

#### 3. Stemming

- 例）: democracy, democratic, and democratization -> democr
- 語幹を抽出する

#### 4. Stopwords

- モデルにとって重要な情報を含まない単語
    - 1. 重要ではない単語: 冠詞や前置詞
    - 2. タスクの解決に役立たないようなよくある単語
- NLTK (Natural Language ToolKit library for Python): ストップワードが予め定義されたリストを持つ
- sklearn.feature_extraction.text.CountVectorizer: max_df（単語の閾値(?)を設定できる）

### まとめ

- BOW 適用のパイプライン
    - 1. 前処理: 小文字化・ステミング・レンマ化・ストップワード除去
    - 2. Ngrams: 局所的な文脈を捉えることに役立ちうる
    - 3. 後処理: TFiDF

## Word2vec, CNN

### Word2vec

- Word2Vec は BOW と同様に単語や文章をベクトル化することを目指すが、もう少し正確に行う。
- 異なる単語が同じ文脈で頻繁に用いられる場合、それらのベクトル空間上での距離は近いという発想を基本とする。
    - king + woman - man = queen という有名な例のように、ベクトル化することで単語同士の足し算・引き算ができる。
    - ベクトルの方向に意味を与えることができる。
- 実装方法としては以下
    - Words: Word2vec, Glove(Glocal Vector for word representation), FastText, etc.
    - Sentences: Doc2vec, etc.
        - 文章のベクトル化の方法には、単語ベクトルの平均・合計を計算する方法もある。
    - 事前学習モデルが有る: Wikipedia
    - ちなみに、Word2vec の学習にはラベルは不要で、各単語の文脈を抽出するためのテキストが必要なだけである。
    
### BOW と w2v の比較

- 1. Bag of words
    - a. 大きなベクトル
    - b. ベクトルの各値の意味は解釈可能
- 2. Word2vec
    - a. 比較的小さなベクトル
    - b. ベクトルの値はある場合にのみ解釈可能なことがある
    - c. 同じような意味の単語はしばしば同じように埋め込みされる

### Image -> Vector

- 1. Descriptors
- 2. スクラッチでモデル構築
- 3. Fine-tuning

## CNN

### CNN
- CNN は、単語に対して Word2vec で埋め込み表現を獲得するのと同様に、画像に対して圧縮された表現を獲得する。
- 画像から得られた何らかの出力を Descriptors と呼ぶ

### Fine-tuning
- Descriptors の最初の層は入力に依存するところが大きいが、後の方ほど入力に依存しにくくなる。
- （この性質を利用して、）事前学習モデルをタスクに適したものにチューニングすることがある。
    - この処理を fine-tuning と呼ぶ。
    - 特に小さなデータセットを使用する場合や同じようなタスクの場合、事前学習モデルを fine-tuning した方がスクラッチで学習したモデルよりも良い事が多い。
- fine-tuning の例: 出力の最終層を除いて、タスクに適した出力層に変更する（その他の重みパラメータは変えない）

### Augumentation
- 学習データを増やしたいときに行なう。
- 元の画像を回転（90度/180度/270度 etc.）させたりする。
- モデルをロバストにし、過学習を避けるのに役立つ。
- モデルを作る際に、学習データと検証データの分離に気をつける必要がある。

### まとめ
- a. 異なる層から特徴を抽出することができる。
- b. 事前学習モデルの注意深い選択が役に立つことがある。
- c. Fine-tuning は事前学習モデルを強化させることができる。
- d. Data Augmentation はモデルを改善しうる。

## リンク

### Feature extraction from text with Sklearn

#### Bag of words
- [Feature extraction from text with Sklearn](https://scikit-learn.org/stable/modules/feature_extraction.html)
- [More examples of using Sklearn](https://andhint.github.io/machine-learning/nlp/Feature-Extraction-From-Text/)
#### Word2vec
- [Tutorial to Word2vec](https://www.tensorflow.org/tutorials/representation/word2vec)
- [Tutorial to word2vec usage](https://rare-technologies.com/word2vec-tutorial/)
- [Text Classification With Word2Vec](http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/)
- [Introduction to Word Embedding Models with Word2Vec](https://taylorwhitten.github.io/blog/word2vec)
#### NLP Libraries
- [NLTK](http://www.nltk.org/)
- [TextBlob](https://github.com/sloria/TextBlob)

### Feature extraction from images
#### Pretrained models
- [Using pretrained models in Keras](https://keras.io/applications/)
- [Image classification with a pre-trained deep neural network](https://www.kernix.com/blog/image-classification-with-a-pre-trained-deep-neural-network_p11)
#### Finetuning
- [How to Retrain Inception's Final Layer for New Categories in Tensorflow](https://www.tensorflow.org/hub/tutorials/image_retraining)
- [Fine-tuning Deep Learning Models in Keras](https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html)